# Function defintions

In [1]:
class ecosystem_services:
    # ---------------------------------------------------------------------------------
    # Global variables
    # ---------------------------------------------------------------------------------
    # Shapefiles
    dir_data='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/'
    ES_shp = f'{dir_data}shapefiles/grid_WGS84_ES/grid_WGS84_ES/grid_WGS84_ES.shp'

    # Netcdf files
    gridDir = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
    grid_nc = f'{gridDir}grid_lake_michigan.nc'

    # Variables
    service_names = ['FLICKR', 'BIRDVISITS', 'BOATLPARK', 
                     'MARSLIPS', 'PARK_VISIT', 'WATWITHDWL']

    # ---------------------------------------------------------------------------------
    # Describe the services
    # ---------------------------------------------------------------------------------
    def description(self):
        """
        description()
            description of ecosystem services
        """
        description = """
            Service      Description
            -------------------------------------------------
            GRID_FID     Matches FID_1 in original shapefile
            BEACHES      Number of beaches
            FLICKR       Number of FLICKR photos at beaches
            BIRDHTSPT    Number of e-bird birding hotspots
            BIRDVISITS   Number of logged visits to hotspots
            BOATLAUNCH   Number of boat launches
            BOATLPARK    Number of boat launch parking spaces
            MARINAS      Number of marinas
            MARSLIPS     Number of marina slips
            PARKS        Number of parks
            PARK_AREA    Park area in 2km cell (square kilometers)
            PARK_VISIT   Area-weighted annual park visitation
            WATINTAKE    Number of municipal water intake pipes
            WATWITHDWL   Water withdrawals in millions of gallons/year
            """
        # Display description
        print(description)

    # ---------------------------------------------------------------------------------
    # normalize data
    # ---------------------------------------------------------------------------------
    def normalize(self, data):
        """
        normalize(data
            normalized data to be between 0-1
        """
        # Log10 transform to get data on similar scale
        data_tran = np.log10(data + 1)

        # Scale between 0 - 1
        return (data_tran - np.min(data_tran)) / (np.max(data_tran) - np.min(data_tran))

    # ---------------------------------------------------------------------------------
    # Distance between two lat/lon points
    # ---------------------------------------------------------------------------------
    def haversine_distance(self, latitude_1, longitude_1, latitude_2, longitude_2):
        """
        haversine_distance(latitude_1, longitude_1, latitude_2, longitude_2)
            Calculate the great circle distance between two points
            on the earth (specified in decimal degrees)
        """
        EARTH_RADIUS = 6371

        dlon = np.radians(longitude_2 - longitude_1)
        dlat = np.radians(latitude_2 - latitude_1)
        lon1 = np.radians(longitude_1)
        lon2 = np.radians(longitude_2)
        lat1 = np.radians(latitude_1)
        lat2 = np.radians(latitude_2)

        a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        
        ## distance in KM
        return EARTH_RADIUS * c
    
    # ---------------------------------------------------------------------------------
    # get the model grid
    # ---------------------------------------------------------------------------------
    def get_model_grid(self):
        """
        get_model_grid()
            outputs dictionary of grid parameters:

            SA      surface area of each cell (m2)
            depth   depth of each cell (m)
            lon     longitude (deg)
            lat     latitude (deg)
            drF     r cell face separation (m)
            rA      face area at cell (m)
            dlon    diff in lon (deg)
            dlat    diff in lat (deg)
            volCell volume of surface cells (m3)
            volLake volume of lake (m3)
        """
        # Read grid file
        ds = xr.open_dataset(self.grid_nc)

        # Variables
        SA = ds['rA'].values
        depth = ds['Depth'].values
        lon = ds['X'].values
        lat = ds['Y'].values
        drF = ds['drF'].values
        rA = ds['rA'].values
        dlon = np.diff(lon[0:2]) / 2
        dlat = np.diff(lat[0:2]) / 2
        volCell = rA * drF[0]
        vol = SA * depth
        volLake = np.sum(vol.flat)

        # output
        out = {'SA': SA,
               'depth': depth,
               'lon': lon,
               'lat': lat,
               'drF': drF,
               'rA': rA,
               'dlon': dlon,
               'dlat': dlat,
               'volCell': volCell,
               'vol': vol,
               'volLake': volLake}

        return out

    # ---------------------------------------------------------------------------------
    # shift phos to ES map
    # ---------------------------------------------------------------------------------
    def shift_to_service(self, service_map, phosphorous_map, width=3):
        """
        shift_to_service(service_map, phos_map)
             INPUT
             service_map        2D map of ecosystem services
             phosphorous_map    2D of phosphorous

             OUTPUT
             phosphorous_ES     phosphorous mapped to ES map

             this function finds the closest water point to ES
             and puts that phosphorous value in its place
        """
        # dimension of ecosystem service map (i.e. LM map)
        # shape[0] = 276 @lat, shape[1] = 200 @lon
        shape = service_map.shape

        # mask where there are ES values
        mask_es = (service_map != 0)

        # Initialize matrix for phos data
        phos_es = np.zeros((shape[0], shape[1]))

        # Read model grid
        grid = self.get_model_grid()
        lat = grid['lat']
        lon = grid['lon']
        depth = grid['depth']

        # create meshgrid
        lon_grid, lat_grid = np.meshgrid(lon, lat)

        # make binary called water (1 = water,  0 = land)
        water = np.copy(depth)
        water[(water != 0)] = 1
        mask_water = (water == 1)

        # Loop over grid
        for i in range(shape[0]):
            for j in range(shape[1]):
                # Test if point is an ES point
                if (mask_es[i, j] == True):
                    # Define a box around ES point
                    if (i - width < 0):
                        lat_inds = np.arange(0, i + width)
                        lon_inds = np.arange(j - width, j + width)
                    elif (j + 3 >= shape[1] - 1):
                        lat_inds = np.arange(i - width, i + width)
                        lon_inds = np.arange(j - width, shape[1])
                    else:
                        lat_inds = np.arange(i - width, i + width)
                        lon_inds = np.arange(j - width, j + width)

                    # Meshrid the indices
                    X, Y = np.meshgrid(lat_inds, lon_inds)

                    # Find Water points
                    water_box = water[X, Y]

                    # Get grid if totally on land
                    # keep that point as zero and break out of loop
                    if (np.sum(water_box.ravel()) == 0):
                        break

                    # Calculate Distance
                    dis = self.haversine_distance(lat[X], lon[Y], lat[i], lon[j])
                    dis_min = np.min(dis[(water_box == 1)])

                    # find indices in small box
                    ind_box = np.where(dis == dis_min)
                    iBox = ind_box[0][0]
                    jBox = ind_box[1][0]
                    #print(iNew, jNew)

                    # New location of point
                    iNew = X[iBox, jBox]
                    jNew = Y[iBox, jBox]

                    # Put closest phosphorous point in location of ES point
                    phos_es[i, j] = phosphorous_map[iNew, jNew].copy()

        return phos_es

In [2]:
def normalize(data):
    """
    normalize(data
        normalized data to be between 0-1
    """
    # Log10 transform to get data on similar scale
    data_tran = np.log10(data + 1)

    # Scale between 0 - 1
    return (data_tran - np.min(data_tran)) / (np.max(data_tran) - np.min(data_tran))

In [3]:
def calculate_csd(ex_days, es_norm):
    """ 
    cses_val = cses(P_CONC_MAP, NORMALIZED_ES)
    calculate CSES associated with each service 
    then you can sum them all up to get total
    """
    # This function uses ecosystem service class
    es = ecosystem_services()
    
    # Shift exceedance days to service location
    ex_days_shifted = es.shift_to_service(es_norm, ex_days)

    # Calculate CSD
    return np.nansum(ex_days_shifted.flatten() * es_norm.flatten())



In [4]:
def ed_shifted(ex_days, es_norm):
    """ 
    
    """
    # This function uses ecosystem service class
    es = ecosystem_services()
    
    # Shift exceedance days to service location
    return es.shift_to_service(es_norm, ex_days)

